In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function

In [2]:

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = 'IEEE_13Node_Modified_01.dss'
dss.run_command('Redirect ' + fn)

''

Vectorized Solver

In [3]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE13_splitmultiphase.dss'
#fn = 'IEEE13_nominalload_vec.dss'
#fn = 'IEEE13_capnominalvoltage.dss'
fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'hello.dss'
#fn = '02node_threephase_unbalanced_unchanged.dss' #original
#fn = '02node_threephase_unbalanced.dss' #apq = .9, az = .1
#fn = '06node_threephase_unbalanced.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


# Pre-generate matrices
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None)

for i in range(len(times)):
    # Run NR3 with variations in time, DER, and capacitance
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i])    
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91821246-0.10957476j  0.94941713-0.05560837j
   0.        +0.j          0.        +0.j          0.90898503-0.11336917j
   0.90169707-0.11722609j  0.        +0.j          0.91005304-0.10758234j
   0.94379241-0.07218327j  0.99789667-0.00254538j  0.95583011-0.05345307j
   0.91821246-0.10957476j  0.95262362-0.05453072j  0.91603083-0.10972829j]
 [-0.5       -0.8660254j  -0.53449743-0.85186265j -0.51752657-0.83897394j
  -0.51336593-0.83343754j -0.51205107-0.83131772j -0.53898377-0.85237005j
  -0.54286784-0.85266959j  0.        +0.j          0.        +0.j
  -0.52311502-0.84534436j -0.50086179-0.8649375j  -0.51809758-0.84317945j
  -0.53449743-0.85186265j -0.51781207-0.84107669j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.38105732+0.80661364j -0.42607817+0.830691j
  -0.42587663+0.83523106j -0.4241427 +0.83395432j -0.3800959

Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91584797-0.11181557j  0.94802244-0.05674797j
   0.        +0.j          0.        +0.j          0.90638127-0.11562998j
   0.89889647-0.11951289j  0.        +0.j          0.90751482-0.10976647j
   0.94217709-0.07366588j  0.9978368 -0.00259777j  0.95457275-0.05455315j
   0.91584797-0.11181557j  0.9512976 -0.05565056j  0.91361897-0.11197639j]
 [-0.5       -0.8660254j  -0.53505305-0.85159845j -0.51779706-0.83845256j
  -0.51354736-0.83280506j -0.51220418-0.83064409j -0.53959827-0.85210783j
  -0.54352789-0.85240546j  0.        +0.j          0.        +0.j
  -0.52347811-0.84494991j -0.50087526-0.86491652j -0.51838036-0.84273891j
  -0.53505305-0.85159845j -0.51808871-0.84059573j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.37832257+0.80431897j -0.42440053+0.82938211j
  -0.42419227+0.83401717j -0.42242242+0.83270986j -0.37735477+0.80087933j
  -0.37645

Non-vectorized solver reading OpenDSS files

In [4]:
fn= 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
#fn = 'IEEE13_splitmultiphase.dss'
#fn = 'IEEE13_nominalload_vec.dss'
#fn = 'IEEE13_capnominalvoltage.dss'
#fn = 'hello.dss'
fn = '02node_threephase_unbalanced_unchanged.dss' #original
#fn = '02node_threephase_unbalanced.dss' #aPQ = 0.9 aZ = 0.1
#fn = '06node_threephase_unbalanced.dss' #model = 8
fn = '06node_threephase_unbalanced.dss' 
fn = 'IEEE_13Node_Modified_01.dss'

VNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR02 =  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR02 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR02 = np.zeros((len(times), 3, nnode), dtype = "complex")
spah = np.zeros((len(times), 3, nnode), dtype = "complex") #store load values

for i in range(len(times)):
    network1 = nmf2(fn, times[i])

    spah[i, :, :] = network1.loads.spu

    nnode = network1.nodes.nnode
    nline = network1.lines.nline

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    network1.loads.aPQ =1.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aI = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aZ = 0.0*np.ones((3,nnode))*(network1.loads.spu != 0)

    network1.loads.spu = 1*network1.loads.spu

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    VNR02[i, :, :] = np.reshape(VNR, (3, nnode))
    INR02[i, :, :] = np.reshape(INR, (3, nline))
    STXNR02[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR02[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR02[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR02[i, :, :] = np.reshape(sNR, (3, nnode))
    print('VNR\n', np.round(VNR,decimals=6))
    print('INR\n', np.round(INR,decimals=6))
    print('STXNR\n', np.round(STXNR,decimals=6))
    print('SRXNR\n', np.round(SRXNR,decimals=6))
    print('iNR\n', np.round(iNR,decimals=6))
    print('sNR\n', np.round(sNR,decimals=6))
    print('itercount:', itercount)

    print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
    print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
    print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

BASE
Vbase: 2401.7771198288433
Sbase: 1000000.0
Ibase: 416.358367204057
Zbase: 5.768533333333334

NODES
nnode: 15
nodelist: ['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11  9  1 12 13  2  3  4  5  6 14  7  8]
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'ac', 'c', 'a']
PH:
 [[1 1 1 1 1 1 1 0 0 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0]]
config: ['mtx60

0.2236787787715529
5.9358574004244
0.0051404373025853035
egienvalues
1.503331340866121
0.21968090306685933
5.944315756575398
0.004300627611222102
egienvalues
1.5032319115375483
0.2195507895107291
5.944334834969342
0.004283514251077435
egienvalues
1.5032318836354082
0.21955074782766365
5.94433475457407
0.004283506050328909
VNR
 [[ 1.      +0.j        0.918212-0.109575j  0.949417-0.055608j
   0.      +0.j        0.      +0.j        0.908985-0.113369j
   0.901697-0.117226j  0.      +0.j        0.910053-0.107582j
   0.943792-0.072183j  0.997897-0.002545j  0.95583 -0.053453j
   0.918212-0.109575j  0.952624-0.054531j  0.916031-0.109728j]
 [-0.5     -0.866025j -0.534497-0.851863j -0.517527-0.838974j
  -0.513366-0.833438j -0.512051-0.831318j -0.538984-0.85237j
  -0.542868-0.85267j   0.      +0.j        0.      +0.j
  -0.523115-0.845344j -0.500862-0.864938j -0.518098-0.843179j
  -0.534497-0.851863j -0.517812-0.841077j  0.      +0.j      ]
 [-0.5     +0.866025j -0.381057+0.806614j -0.426078+0.83

0.23785455392813354
5.586580452161168
0.01753981081089471
egienvalues
1.508609077447245
0.22322333967594105
5.94186979645273
0.005060794470640713
egienvalues
1.5028379520976076
0.21901298457693053
5.950875639379964
0.004211972174529017
egienvalues
1.50273306199294
0.2188749088279049
5.950898249037093
0.0041942348634469
egienvalues
1.5027330298900625
0.21887486122069616
5.950898163544475
0.004194225919102247
VNR
 [[ 1.      +0.j        0.917027-0.1107j    0.948718-0.056181j
   0.      +0.j        0.      +0.j        0.90768 -0.114504j
   0.900293-0.118375j  0.      +0.j        0.908781-0.108679j
   0.942983-0.072928j  0.997867-0.002572j  0.9552  -0.054005j
   0.917027-0.1107j    0.951959-0.055093j  0.914822-0.110857j]
 [-0.5     -0.866025j -0.534776-0.851729j -0.517662-0.838711j
  -0.513457-0.833119j -0.512128-0.830979j -0.539292-0.852237j
  -0.543199-0.852536j  0.      +0.j        0.      +0.j
  -0.523297-0.845145j -0.500869-0.864927j -0.518239-0.842957j
  -0.534776-0.851729j -0.517951

0.01753981081089471
egienvalues
1.5082424615491812
0.2227740980125073
5.9478574960123405
0.004982929943026741
egienvalues
1.502350810981152
0.2183469890017395
5.9574316423071885
0.004125350123522535
egienvalues
1.5022402747479324
0.21820060079993134
5.957458970567804
0.004106981742120977
egienvalues
1.502240237937422
0.21820054655236704
5.957458880501365
0.004106972002110871
VNR
 [[ 1.      +0.j        0.915848-0.111816j  0.948022-0.056748j
   0.      +0.j        0.      +0.j        0.906381-0.11563j
   0.898896-0.119513j  0.      +0.j        0.907515-0.109766j
   0.942177-0.073666j  0.997837-0.002598j  0.954573-0.054553j
   0.915848-0.111816j  0.951298-0.055651j  0.913619-0.111976j]
 [-0.5     -0.866025j -0.535053-0.851598j -0.517797-0.838453j
  -0.513547-0.832805j -0.512204-0.830644j -0.539598-0.852108j
  -0.543528-0.852405j  0.      +0.j        0.      +0.j
  -0.523478-0.84495j  -0.500875-0.864917j -0.51838 -0.842739j
  -0.535053-0.851598j -0.518089-0.840596j  0.      +0.j      ]
 [

0.22233532593030278
5.953760393622639
0.004907550257366463
egienvalues
1.5018746200901754
0.21768953369687166
5.96391684942372
0.0040415484574841745
egienvalues
1.5017583100951992
0.21753456196768833
5.963950052542106
0.004022548601535247
egienvalues
1.501758268077926
0.21753450036989996
5.963949958591716
0.004022538019972341
VNR
 [[ 1.      +0.j        0.914686-0.112911j  0.947337-0.057305j
   0.      +0.j        0.      +0.j        0.905102-0.116735j
   0.897521-0.12063j   0.      +0.j        0.906268-0.110834j
   0.941383-0.074391j  0.997807-0.002623j  0.953955-0.055091j
   0.914686-0.112911j  0.950646-0.056198j  0.912434-0.113075j]
 [-0.5     -0.866025j -0.535326-0.851473j -0.51793 -0.8382j
  -0.513636-0.832498j -0.512279-0.830317j -0.5399  -0.851984j
  -0.543852-0.85228j   0.      +0.j        0.      +0.j
  -0.523656-0.84476j  -0.500882-0.864906j -0.518519-0.842526j
  -0.535326-0.851473j -0.518224-0.840363j  0.      +0.j      ]
 [-0.5     +0.866025j -0.376982+0.803189j -0.423579+0

0.2219111496853807
5.959518362927371
0.004835311389132131
egienvalues
1.5014139328485618
0.2170472858799371
5.970263535378062
0.003961295327010334
egienvalues
1.5012917878772774
0.2168835550951086
5.970303686160201
0.003941670677886558
egienvalues
1.501291740184696
0.21688348547503336
5.9703035891604355
0.003941659217448258
VNR
 [[ 1.      +0.j        0.913553-0.113975j  0.94667 -0.057847j
   0.      +0.j        0.      +0.j        0.903855-0.117808j
   0.89618 -0.121715j  0.      +0.j        0.905052-0.11187j
   0.94061 -0.075095j  0.997779-0.002648j  0.953353-0.055614j
   0.913553-0.113975j  0.950011-0.05673j   0.911278-0.114143j]
 [-0.5     -0.866025j -0.535591-0.851354j -0.518059-0.837957j
  -0.513723-0.832202j -0.512353-0.830002j -0.540193-0.851865j
  -0.544167-0.852161j  0.      +0.j        0.      +0.j
  -0.523829-0.844577j -0.500888-0.864897j -0.518654-0.842321j
  -0.535591-0.851354j -0.518356-0.840139j  0.      +0.j      ]
 [-0.5     +0.866025j -0.375678+0.802086j -0.422779+0.

OpenDSS Solution

In [5]:
dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
#dss.run_command('Redirect 02node_threephase_unbalanced_unchanged.dss')
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
#dss.run_command('Redirect IEEE13_splitmultiphase.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced_edit.dss')
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')

VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')
kW_list = np.array([])
kvar_list = np.array([])

# Store the loads
for k in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k])
    kW_list = np.append(kW_list, dss.Loads.kW())
    kvar_list = np.append(kvar_list, dss.Loads.kvar())

for i in range(len(times)):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
    Vbase = dss.Bus.kVBase() * 1000
    Sbase = 1000000.0
    Ibase = Sbase/Vbase
    Zbase = Vbase/Ibase
    
    # Set slack bus (sourcebus) voltage reference in p.u.
    SlackBusVoltage = 1.000
    dss.Vsources.PU(SlackBusVoltage)
    
    # Time-varying load
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
        dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
      
    dss.Solution.Convergence(0.000000000001)
    
    # Solve power flow with OpenDSS file
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        # set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        # Easy process to get all names and count of loads, a trick to avoid
        # some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')
    
    
    for k in range(len(dss.Loads.AllNames())):
        dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0]) #bus name for load
        idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) #what index is bus at in allbusnames()
        load_ph_arr_temp = [0, 0, 0] #ph of load
        for ph1 in range(1, 4): #update existing ph of load
            pattern = r"\.%s" % (str(ph1))
            load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
            if load_ph:
                load_ph_arr_temp[ph1 - 1] = 1
        n= 0
        for m in range(len(load_ph_arr_temp)):
            if load_ph_arr_temp[m] == 1:
                #print('squirrel')
                load_arr[i, m, idxbs] +=  (dss.CktElement.Powers()[n] + 1j * dss.CktElement.Powers()[n + 1])*1e3/1e6     #where 1e6 is sbase 
                n += 2
    
    # print(dss.Solution.Converged())

    # Print number of buses, and bus names
    print(len(dss.Circuit.AllBusNames()))
    print(dss.Circuit.AllBusNames())

    # Print number of loads, and load names
    print(len(dss.Loads.AllNames()))
    print(dss.Loads.AllNames())

    print('')

    VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

    for k1 in range(len(dss.Circuit.AllBusNames())):
        
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
    #     print(dss.Circuit.AllBusNames()[k1])
    #     print(dss.Bus.Nodes())

    #     print('puVOTLAGES - LN CARTESIAN')
    #     print(dss.Bus.PuVoltage())

        ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1
        
        Vtemp = np.asarray(dss.Bus.PuVoltage())

        Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]
        

    #     print(np.asarray(dss.Bus.Nodes(),'int'))

        VDSS[ph,k1] = Vtemp    

    #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


    #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
    #     for k2 in range(len(dss.Bus.Nodes())):
    #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))


    IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
        Imn = np.asarray(dss.CktElement.Currents())/Ibase
    #     print(Imn)
        Imn = Imn[0:int(len(Imn)/2)]
    #     print(Imn)
        Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
    #     print(Imn)
        IDSS[ph,k1] = Imn
    #     print('')

    print('IDSS\n', np.round(IDSS,decimals=6))

    STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
    SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
    #     print(dss.CktElement.BusNames())
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    #     print(ph)
        Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
    #     print(Sk)

    #     print(Sk[0:int(len(Sk)/2)])
    #     print(Sk[int(len(Sk)/2):])

        STXtemp = Sk[0:int(len(Sk)/2)]
        SRXtemp = Sk[int(len(Sk)/2):]

        STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
    #     print(STXtemp)

        SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])
    #     print(SRXtemp)

        STXDSS[ph,k1] = STXtemp
        SRXDSS[ph,k1] = SRXtemp

    VDSS0[i, :, :] = VDSS
    IDSS0[i, :, :] = IDSS
    SRXDSS0[i, :, :] = SRXDSS
    STXDSS0[i, :, :] = STXDSS
    
    
    print('STXDSS\n', np.round(STXDSS,decimals=6))
    print('SRXDSS\n', np.round(SRXDSS,decimals=6))

    print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
    print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
    print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  15
Tolerance:  1e-12

15
['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
15
['671', '634a', '634b', '634c', '645', '646', '692', '675a', '675b', '675c', '611', '652', '670a', '670b', '670c']

VDSS
 [[ 0.999843-1.60000e-04j  0.912516-1.12603e-01j  0.946419-5.72680e-02j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.902754-1.15920e-01j
   0.89494 -1.19303e-01j  0.      +0.00000e+00j  0.904324-1.10687e-01j
   0.939912-7.43020e-02j  0.997605-2.77700e-03j  0.952849-5.51130e-02j
   0.912516-1.12603e-01j  0.949634-5.61910e-02j  0.910328-1.12870e-01j]
 [-0.500057-8.65881e-01j -0.537225-8.55988e-01j -0.518925-8.41096e-01j
  -0.514713-8.35605e-01j -0.513383-8.33500e-01j -0.542023-8.56426e-01j
  -0.546219-8.56652e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
  -0.524949-8.48126e-01j -0.500982-8.64900e-01j -0.519495-8.45281e-01j
  -

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [6]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 
    print(np.max(np.abs(VDSS0[i] - VNR02[i])))

    print('Complex Current difference')
    print(np.max(np.abs(INR01[i] - INR02[i])))
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    print(np.max(np.abs(IDSS0[i] - INR02[i])))

    print('Complex TX Power difference')
    print(np.max(np.abs(STXNR01[i] - STXNR02[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR02[i])))

    print('Complex RX Power difference')
    print(np.max(np.abs(SRXNR01[i] - SRXNR02[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR02[i])))

    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VNR01[i]) - np.abs(VNR02[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR02[i]))))

    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VNR01[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print("\n")

Timestep 0 

Complex Voltage difference
2.482534153247273e-16
0.011412453789056105
0.011412453789055995
Complex Current difference
2.482534153247273e-16
0.08021579510050816
0.08021579510050798
Complex TX Power difference
2.482534153247273e-16
0.07984871733584416
0.07984871733584402
Complex RX Power difference
2.482534153247273e-16
0.07954128756370833
0.0795412875637082
Voltage Magnitude difference
2.220446049250313e-16
0.010598211246176326
0.010598211246176326
Voltage Angle difference
2.842170943040401e-14
0.3116124692526796
0.3116124692526796


Timestep 1 

Complex Voltage difference
2.482534153247273e-16
0.01161374439199143
0.011613744391991318
Complex Current difference
7.021666937153402e-16
0.08169510392962899
0.08169510392962893
Complex TX Power difference
7.021666937153402e-16
0.08131856623162231
0.08131856623162224
Complex RX Power difference
6.753223014464259e-16
0.08099772121779872
0.08099772121779866
Voltage Magnitude difference
2.220446049250313e-16
0.010799646277636632
0.01

Load comparison between non-vectorized and OpenDSS

In [7]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

Timestep 0 
 Magnitude of the Difference: 

[[0.00000000e+00 2.70384930e-11 6.42635962e-12 0.00000000e+00
  0.00000000e+00 5.32391906e-12 3.45714899e-11 0.00000000e+00
  9.79355018e-12 8.12026153e-13 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.36374656e-11 2.68471117e-12 3.77505624e-12
  2.39611839e-12 0.00000000e+00 2.81727322e-12 0.00000000e+00
  0.00000000e+00 1.66024492e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.01786324e-11 3.60895858e-12 0.00000000e+00
  3.11775754e-12 7.25589934e-12 1.72513228e-11 8.79027907e-12
  0.00000000e+00 4.14188786e-12 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]]
Max difference: 
 
3.457148992939218e-11


Timestep 1 
 Magnitude of the Difference: 

[[0.00000000e+00 3.96133662e-13 9.40253225e-14 0.00000000e+00
  0.00000000e+00 6.72506085e-14 5.07442950e-13 0.00000000e+00
  1.43327653e-13 1.18911922e-14 0.00000000

---